# Backtesting a strategy on Bitcoin

In [3]:
from datetime import datetime
from os import path

import backtrader as bt
import pandas as pd
import requests

output_folder = "data/bitmex"

/Users/r.orac/envs/r/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [4]:
date_from = "2020-10-01"
date_to = "2020-10-31"
dates = pd.date_range(date_from, date_to).astype(str).str.replace("-", "")
dates

Index(['20201001', '20201002', '20201003', '20201004', '20201005', '20201006',
       '20201007', '20201008', '20201009', '20201010', '20201011', '20201012',
       '20201013', '20201014', '20201015', '20201016', '20201017', '20201018',
       '20201019', '20201020', '20201021', '20201022', '20201023', '20201024',
       '20201025', '20201026', '20201027', '20201028', '20201029', '20201030',
       '20201031'],
      dtype='object')

In [5]:
dataset_filename = path.join(output_folder, f"XBT_USD_{date_from}_{date_to}.csv")
dataset_filename

'data/bitmex/XBT_USD_2020-10-01_2020-10-31.csv'

## Download data

In [ ]:
BASE_URL = "https://s3-eu-west-1.amazonaws.com/public.bitmex.com/data/trade/%s"


def download_trade_file(filename, output_folder):
    print(f"Downloading {filename} file")
    url = BASE_URL % filename
    resp = requests.get(url)
    if resp.status_code != 200:
        print(f"Cannot download the {filename} file. Status code: {resp.status_code}")
        return
    with open(path.join(output_folder, filename), "wb") as f:
        f.write(resp.content)
    print(f"{filename} downloaded")

In [ ]:
for date in dates:
    filename = date + ".csv.gz"
    download_trade_file(filename, output_folder)
print("done")

## Preprocess data

In [28]:
import glob

filepaths = glob.glob(path.join(output_folder, "*.csv.gz"))
filepaths = sorted(filepaths)
filepaths

['data/bitmex/20201001.csv.gz',
 'data/bitmex/20201002.csv.gz',
 'data/bitmex/20201003.csv.gz',
 'data/bitmex/20201004.csv.gz',
 'data/bitmex/20201005.csv.gz',
 'data/bitmex/20201006.csv.gz',
 'data/bitmex/20201007.csv.gz',
 'data/bitmex/20201008.csv.gz',
 'data/bitmex/20201009.csv.gz',
 'data/bitmex/20201010.csv.gz',
 'data/bitmex/20201011.csv.gz',
 'data/bitmex/20201012.csv.gz',
 'data/bitmex/20201013.csv.gz',
 'data/bitmex/20201014.csv.gz',
 'data/bitmex/20201015.csv.gz',
 'data/bitmex/20201016.csv.gz',
 'data/bitmex/20201017.csv.gz',
 'data/bitmex/20201018.csv.gz',
 'data/bitmex/20201019.csv.gz',
 'data/bitmex/20201020.csv.gz',
 'data/bitmex/20201021.csv.gz',
 'data/bitmex/20201022.csv.gz',
 'data/bitmex/20201023.csv.gz',
 'data/bitmex/20201024.csv.gz',
 'data/bitmex/20201025.csv.gz',
 'data/bitmex/20201026.csv.gz',
 'data/bitmex/20201027.csv.gz',
 'data/bitmex/20201028.csv.gz',
 'data/bitmex/20201029.csv.gz',
 'data/bitmex/20201030.csv.gz',
 'data/bitmex/20201031.csv.gz']

In [125]:
df_list = []

for filepath in filepaths:
    print(f"Reading {filepath}")
    df_ = pd.read_csv(filepath)
    df_ = df_[df_.symbol == "XBTUSD"]
    print(f"Read {df_.shape[0]} rows")
    df_list.append(df_)

df = pd.concat(df_list)

df_list = None
df_ = None

df.shape

Reading data/bitmex/20201001.csv.gz
Read 294461 rows
Reading data/bitmex/20201002.csv.gz
Read 239015 rows
Reading data/bitmex/20201003.csv.gz
Read 125868 rows
Reading data/bitmex/20201004.csv.gz
Read 131439 rows
Reading data/bitmex/20201005.csv.gz
Read 145599 rows
Reading data/bitmex/20201006.csv.gz
Read 202069 rows
Reading data/bitmex/20201007.csv.gz
Read 126020 rows
Reading data/bitmex/20201008.csv.gz
Read 222291 rows
Reading data/bitmex/20201009.csv.gz
Read 170074 rows
Reading data/bitmex/20201010.csv.gz
Read 229706 rows
Reading data/bitmex/20201011.csv.gz
Read 168194 rows
Reading data/bitmex/20201012.csv.gz
Read 257813 rows
Reading data/bitmex/20201013.csv.gz
Read 218621 rows
Reading data/bitmex/20201014.csv.gz
Read 198014 rows
Reading data/bitmex/20201015.csv.gz
Read 213864 rows
Reading data/bitmex/20201016.csv.gz
Read 221080 rows
Reading data/bitmex/20201017.csv.gz
Read 119530 rows
Reading data/bitmex/20201018.csv.gz
Read 115645 rows
Reading data/bitmex/20201019.csv.gz
Read 24032

(7784790, 10)

In [126]:
df.head()

,timestamp,symbol,side,size,price,tickDirection,trdMatchID,grossValue,homeNotional,foreignNotional
139251,2020-10-01D00:00:02.554962000,XBTUSD,Buy,44,10778.5,ZeroPlusTick,885ff3f5-7126-a6fc-8d03-d38a9fb4102f,408232,0.004082,44.0
139252,2020-10-01D00:00:02.558276000,XBTUSD,Buy,211,10778.5,ZeroPlusTick,37ee831c-ddbe-fbed-7d54-08dc7cc98645,1957658,0.019577,211.0
139253,2020-10-01D00:00:02.567837000,XBTUSD,Buy,1162,10778.5,ZeroPlusTick,3187b1d7-78c6-f553-a1fa-29b9d28e2d31,10781036,0.107810,1162.0
139254,2020-10-01D00:00:02.574834000,XBTUSD,Buy,42,10778.5,ZeroPlusTick,81c5eafd-a13a-e037-8f8e-1c7acab601f5,389676,0.003897,42.0
139255,2020-10-01D00:00:02.685716000,XBTUSD,Buy,66,10778.5,ZeroPlusTick,878f47af-e7f5-6952-9b29-f13ffaa556f8,612348,0.006123,66.0


In [62]:
df.loc[:, "Datetime"] = pd.to_datetime(df.timestamp.str.replace("D", "T"))

In [65]:
df.drop("timestamp", axis=1, inplace=True)

In [66]:
df.head()

,symbol,side,size,price,tickDirection,trdMatchID,grossValue,homeNotional,foreignNotional,Datetime
139251,XBTUSD,Buy,44,10778.5,ZeroPlusTick,885ff3f5-7126-a6fc-8d03-d38a9fb4102f,408232,0.004082,44.0,2020-10-01 00:00:02.554962
139252,XBTUSD,Buy,211,10778.5,ZeroPlusTick,37ee831c-ddbe-fbed-7d54-08dc7cc98645,1957658,0.019577,211.0,2020-10-01 00:00:02.558276
139253,XBTUSD,Buy,1162,10778.5,ZeroPlusTick,3187b1d7-78c6-f553-a1fa-29b9d28e2d31,10781036,0.107810,1162.0,2020-10-01 00:00:02.567837
139254,XBTUSD,Buy,42,10778.5,ZeroPlusTick,81c5eafd-a13a-e037-8f8e-1c7acab601f5,389676,0.003897,42.0,2020-10-01 00:00:02.574834
139255,XBTUSD,Buy,66,10778.5,ZeroPlusTick,878f47af-e7f5-6952-9b29-f13ffaa556f8,612348,0.006123,66.0,2020-10-01 00:00:02.685716


In [67]:
df = df.groupby(pd.Grouper(key="Datetime", freq="1Min")).agg(
    {"price": ["first", "max", "min", "last"], "foreignNotional": "sum"}
)

In [68]:
df.columns = ["Open", "High", "Low", "Close", "Volume"]

In [69]:
df.loc[:, "OpenInterest"] = 0.0

In [70]:
df = df[df.Close.notnull()]
df.shape

(44629, 6)

In [72]:
df.reset_index(inplace=True)

In [75]:
df.loc[:, "Datetime"] = df["Datetime"].astype(str).str.replace(" ", "T")

In [76]:
df.head()

,Datetime,Open,High,Low,Close,Volume,OpenInterest
0,2020-10-01T00:00:00,10778.5,10783.5,10778.0,10783.5,2283372.0,0.0
1,2020-10-01T00:01:00,10783.0,10805.5,10783.0,10798.5,12216553.0,0.0
2,2020-10-01T00:02:00,10798.0,10798.5,10798.0,10798.0,1290372.0,0.0
3,2020-10-01T00:03:00,10797.5,10800.0,10791.0,10797.5,2017493.0,0.0
4,2020-10-01T00:04:00,10797.5,10826.5,10797.0,10813.5,7358318.0,0.0


In [82]:
df.to_csv(dataset_filename, index=False)
df.shape

(44629, 7)

## Backtest with MACD (only long)

In [114]:
class BitmexComissionInfo(bt.CommissionInfo):
    params = (
        ("commission", 0.00075),
        ("mult", 1.0),
        ("margin", None),
        ("commtype", None),
        ("stocklike", False),
        ("percabs", False),
        ("interest", 0.0),
        ("interest_long", False),
        ("leverage", 1.0),
        ("automargin", False),
    )

    def getsize(self, price, cash):
        """Returns fractional size for cash operation @price"""
        return self.p.leverage * (cash / price)

In [119]:
class MACD(bt.Strategy):
    params = (
        ("macd1", 12),
        ("macd2", 26),
        ("macdsig", 9),
        # Percentage of portfolio for a trade. Something is left for the fees
        # otherwise orders would be rejected
        ("portfolio_frac", 0.98),
    )

    def __init__(self):
        self.val_start = self.broker.get_cash()  # keep the starting cash
        self.size = None
        self.order = None

        self.macd = bt.ind.MACD(
            self.data,
            period_me1=self.p.macd1,
            period_me2=self.p.macd2,
            period_signal=self.p.macdsig,
        )
        # Cross of macd and macd signal
        self.mcross = bt.ind.CrossOver(self.macd.macd, self.macd.signal)

    def next(self):
        if self.order:
            return  # pending order execution. Waiting in orderbook

        print(
            f"DateTime {self.datas[0].datetime.datetime(0)}, "
            f"Price {self.data[0]:.2f}, Mcross {self.mcross[0]}, "
            f"Position {self.position.upopened}"
        )

        if not self.position:  # not in the market
            if self.mcross[0] > 0.0:
                print("Starting buy order")
                self.size = (
                    self.broker.get_cash() / self.datas[0].close * self.p.portfolio_frac
                )
                self.order = self.buy(size=self.size)
        else:  # in the market
            if self.mcross[0] < 0.0:
                print("Starting sell order")
                self.order = self.sell(size=self.size)

    def notify_order(self, order):
        """Execute when buy or sell is triggered
        Notify if order was accepted or rejected
        """
        if order.alive():
            print("Order is alive")
            # submitted, accepted, partial, created
            # Returns if the order is in a status in which it can still be executed
            return

        order_side = "Buy" if order.isbuy() else "Sell"
        if order.status == order.Completed:
            print(
                (
                    f"{order_side} Order Completed -  Size: {order.executed.size} "
                    f"@Price: {order.executed.price} "
                    f"Value: {order.executed.value:.2f} "
                    f"Comm: {order.executed.comm:.6f} "
                )
            )
        elif order.status in {order.Canceled, order.Margin, order.Rejected}:
            print(f"{order_side} Order Canceled/Margin/Rejected")
        self.order = None  # indicate no order pending

    def notify_trade(self, trade):
        """Execute after each trade
        Calcuate Gross and Net Profit/loss"""
        if not trade.isclosed:
            return
        print(f"Operational profit, Gross: {trade.pnl:.2f}, Net: {trade.pnlcomm:.2f}")

    def stop(self):
        """ Calculate the actual returns """
        self.roi = (self.broker.get_value() / self.val_start) - 1.0
        val_end = self.broker.get_value()
        print(
            f"ROI: {100.0 * self.roi:.2f}%%, Start cash {self.val_start:.2f}, "
            f"End cash: {val_end:.2f}"
        )

In [120]:
cerebro = bt.Cerebro()

cerebro.broker.set_cash(1000)

data = bt.feeds.GenericCSVData(
    dataname=dataset_filename,
    dtformat="%Y-%m-%dT%H:%M:%S",
    timeframe=bt.TimeFrame.Ticks,
)

cerebro.resampledata(data, timeframe=bt.TimeFrame.Minutes, compression=60)

cerebro.addstrategy(MACD)

cerebro.broker.addcommissioninfo(BitmexComissionInfo())

# Add TimeReturn Analyzers for self and the benchmark data
cerebro.addanalyzer(
    bt.analyzers.TimeReturn, _name="alltime_roi", timeframe=bt.TimeFrame.NoTimeFrame
)

cerebro.addanalyzer(
    bt.analyzers.TimeReturn,
    data=data,
    _name="benchmark",
    timeframe=bt.TimeFrame.NoTimeFrame,
)

In [122]:
# Execute
results = cerebro.run()
st0 = results[0]

for alyzer in st0.analyzers:
    alyzer.print()

DateTime 2020-10-02 10:00:00, Price 10441.00, Mcross 0.0, Position 0
DateTime 2020-10-02 11:00:00, Price 10462.00, Mcross 1.0, Position 0
Starting buy order
Order is alive
Order is alive
Buy Order Completed -  Size: 0.0936723379850889 @Price: 10462.5 Value: 980.05 Comm: 0.007350 
DateTime 2020-10-02 12:00:00, Price 10430.00, Mcross 0.0, Position 0.0936723379850889
DateTime 2020-10-02 13:00:00, Price 10485.00, Mcross 0.0, Position 0.0936723379850889
DateTime 2020-10-02 14:00:00, Price 10530.50, Mcross 0.0, Position 0.0936723379850889
DateTime 2020-10-02 15:00:00, Price 10522.50, Mcross 0.0, Position 0.0936723379850889
DateTime 2020-10-02 16:00:00, Price 10505.00, Mcross 0.0, Position 0.0936723379850889
DateTime 2020-10-02 17:00:00, Price 10544.00, Mcross 0.0, Position 0.0936723379850889
DateTime 2020-10-02 18:00:00, Price 10533.00, Mcross 0.0, Position 0.0936723379850889
DateTime 2020-10-02 19:00:00, Price 10541.50, Mcross 0.0, Position 0.0936723379850889
DateTime 2020-10-02 20:00:00, P

In [129]:
cerebro.plot(iplot=False, style="bar")

[[<Figure size 4372x2476 with 6 Axes>]]